In [1]:
from sklearn.preprocessing import StandardScaler
from utils import enrich_data, prepare_training_data
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt  # Import the matplotlib.pyplot module
import os
import pandas as pd
import shutil

from utils_plot import print_plot_model_loss

# Define the path to the data folder
data_folder = 'data/learn'
output_folder = 'training_plots'  # Specify the output folder

# Clean the output folder before each run
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)  # Remove the folder and its contents

# Create the output folder
os.makedirs(output_folder)

# Get a list of all files in the data folder
file_list = [filename for filename in os.listdir(data_folder) if filename.endswith('.csv')]

# Loop through each file
for filename in file_list:
    file_path = os.path.join(data_folder, filename)
    print(file_path)

    # Prepare Data
    df, labeled_data, df_for_training, cols = enrich_data(file_path)
    #print(df_for_training)

    # LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
    # normalize the dataset
    scaler = StandardScaler()
    scaler = scaler.fit(df_for_training)
    df_for_training_scaled = scaler.transform(df_for_training)
    #print('df_for_training', df_for_training)

    ## Prepare training data
    trainX, trainY = prepare_training_data(df_for_training_scaled)
    #print('trainY', trainY)

    # Load model weights if available
    try:
        model = load_model('a_model')
        print("Model loaded successfully.")
    except:
        print("No model found. Training from scratch.")
        model = Sequential()
        model.add(LSTM(64, activation='relu', input_shape=(
            trainX.shape[1], trainX.shape[2]), return_sequences=True))
        model.add(LSTM(32, activation='relu', return_sequences=False))
        model.add(Dropout(0.2))
        model.add(Dense(trainY.shape[1]))
        model.compile(optimizer='adam', loss='mse')

    #model.summary()

    # fit the model
    history = model.fit(trainX, trainY, epochs=10, batch_size=250, validation_split=0.1, verbose=1)
    model.save('a_model')

    # Save the plot as an image file in the output folder
    output_filename = os.path.join(output_folder, os.path.splitext(filename)[0] + '_plot.png')
    print_plot_model_loss(output_filename, history.history['loss'], history.history['val_loss'])  




data/learn\NVDA.csv
Model loaded successfully.
Epoch 1/10
23/23 [==============================] - 2s 28ms/step - loss: 0.0688 - val_loss: 5.9208
Epoch 2/10
23/23 [==============================] - 0s 16ms/step - loss: 0.0634 - val_loss: 6.1564
Epoch 3/10
23/23 [==============================] - 0s 17ms/step - loss: 0.0605 - val_loss: 6.1142
Epoch 4/10
23/23 [==============================] - 0s 16ms/step - loss: 0.0583 - val_loss: 6.3156
Epoch 5/10
23/23 [==============================] - 0s 16ms/step - loss: 0.0571 - val_loss: 6.6084
Epoch 6/10
23/23 [==============================] - 0s 16ms/step - loss: 0.0565 - val_loss: 6.9425
Epoch 7/10
23/23 [==============================] - 0s 16ms/step - loss: 0.0544 - val_loss: 7.2610
Epoch 8/10
23/23 [==============================] - 0s 16ms/step - loss: 0.0605 - val_loss: 5.9556
Epoch 9/10
23/23 [==============================] - 0s 16ms/step - loss: 0.0581 - val_loss: 6.2096
Epoch 10/10
23/23 [==============================] - 0s 17ms/s

INFO:tensorflow:Assets written to: a_model\assets


<Figure size 640x480 with 0 Axes>